In [79]:
from astropy.table import Table, vstack, Column, MaskedColumn, pprint
import numpy as np

import matplotlib 
import matplotlib.pyplot as plt
from astropy.convolution import convolve, Gaussian1DKernel

from astropy.io import fits
from matplotlib.backends.backend_pdf import PdfPages

import os
import requests
import sys
sys.path.append('/Users/s2537809/DESI/desispec/py')
sys.path.append('/Users/s2537809/DESI/desimodel/py')
sys.path.append('/Users/s2537809/DESI/desitarget/py')
sys.path.append('/Users/s2537809/DESI/desiutil/py')

#from desispec.io import read_spectra
import numpy as np
# import DESI related modules - 
from desimodel.footprint import radec2pix      # For getting healpix values
import desispec.io                             # Input/Output functions related to DESI spectra
from desispec import coaddition                # Functions related to coadding the spectra


crossmatch = Table.read('crossmatch_classifications.fits', format = 'fits' )

# Download and Produce Spectra

In [86]:
"""
Note: The lengths for indexing are:

HERG = 184

RQAGN = 814

LERG = 1293

SFG = 6294

"""


Overall_class = 'SFG'
my_table = crossmatch[crossmatch['Overall_class'] == f'{Overall_class}'][11:20]
search_column_name = 'TARGETID'
search_values = np.array(my_table[f'{search_column_name}'])



def select_data(table, column_name, search_values):
    # Get the column data as a numpy array
    column_data = table[column_name]

    SURVEY = []
    PROGRAM = []
    HEALPIX = []
    n_values = []
    Z = []

    for search_value in search_values:
        # Find the indices where the condition is met (value == search_value)
        indices = np.where(column_data == search_value)[0]


        if len(indices) > 0:
            # Find the corresponding Survey, program, healpix, and Z values
            SURVEY.append(table['SURVEY'][indices[0]])
            PROGRAM.append(table['PROGRAM'][indices[0]])
            HEALPIX.append(table['HEALPIX'][indices[0]])
            Z.append(table['Z'][indices[0]])
            HEALPIX_str = str(table['HEALPIX'][indices[0]])
            n_str = HEALPIX_str[:3]
            n_values.append(int(n_str))
        else:
            # Handle the case where the search value was not found
            SURVEY.append(None)
            PROGRAM.append(None)
            HEALPIX.append(None)
            n_values.append(None)
            Z.append(None)
    

    return np.array(SURVEY), np.array(PROGRAM), np.array(HEALPIX), np.array(n_values), np.array(Z), 


SURVEY = select_data(my_table, search_column_name, search_values)[0]
PROGRAM = select_data(my_table, search_column_name, search_values)[1]
HEALPIX = select_data(my_table, search_column_name, search_values)[2]
n = select_data(my_table, search_column_name, search_values)[3]
Z = select_data(my_table, search_column_name, search_values)[4]
print (SURVEY,PROGRAM,HEALPIX, n, Z)

# Create a PDF file to save the plots
pdf_filename = f'DESI-{Overall_class}-plots.pdf'
with PdfPages(pdf_filename) as pdf:

    for n, survey, program, healpix, Z, search_value in zip(n, SURVEY, PROGRAM, HEALPIX, Z, search_values):
        #download the file from the given Target ID
        target_url = f"https://data.desi.lbl.gov/public/edr/spectro/redux/fuji/healpix/{survey}/{program}/{n}/{healpix}/coadd-{survey}-{program}-{healpix}.fits"
        
        # Define the folder name and file name
        folder_name = 'DESI_Downloads'
        file_name = f"coadd-{survey}-{program}-{healpix}.fits"
        print(search_value)


        # Check if the folder exists, and create it if it doesn't
        if not os.path.exists(folder_name):
            os.makedirs(folder_name)

        # Check if the file exists in the folder
        file_path = os.path.join(folder_name, file_name)
        if os.path.exists(file_path):
            print(f"{file_name} already exists in {folder_name}. No need to download.")
        else:
            # File doesn't exist, download it
            try:
                response = requests.get(target_url)
                if response.status_code == 200:
                    with open(file_path, 'wb') as file:
                        file.write(response.content)
                    print(f"{file_name} downloaded to {folder_name}.")
                else:
                    print(f"Failed to download {file_name}. Status code: {response.status_code}")
            except Exception as e:
                print(f"An error occurred while downloading {file_name}: {str(e)}")

        
        
        h_coadd = fits.open(f'{folder_name}/{file_name}')
        #h_coadd.info()

        coadd_obj = desispec.io.read_spectra(f'{folder_name}/{file_name}')
        coadd_tgts = coadd_obj.target_ids().data

        # Selecting the particular spectra of the targetid

        row = (coadd_tgts == search_value)
        coadd_spec = coadd_obj[row] 

   # Plotting this spectra -
        fig, ax = plt.subplots(figsize=(20, 6))

        text_x = 0.95  # 95% across the x-axis (right)
        text_y = 0.95  # 95% up the y-axis (top)

        ax.plot(coadd_spec.wave['b'], coadd_spec.flux['b'][0], color='b', alpha=0.5)
        ax.plot(coadd_spec.wave['r'], coadd_spec.flux['r'][0], color='g', alpha=0.5)
        ax.plot(coadd_spec.wave['z'], coadd_spec.flux['z'][0], color='r', alpha=0.5)
        # Over-plotting smoothed spectra in black for all three arms
        ax.plot(coadd_spec.wave['b'], convolve(coadd_spec.flux['b'][0], Gaussian1DKernel(5)), color='k')
        ax.plot(coadd_spec.wave['r'], convolve(coadd_spec.flux['r'][0], Gaussian1DKernel(5)), color='k')
        ax.plot(coadd_spec.wave['z'], convolve(coadd_spec.flux['z'][0], Gaussian1DKernel(5)), color='k')
        
        ax.text(text_x, text_y, 'z=' + f'{Z}', transform=ax.transAxes, ha='right', va='top')
        ax.set_xlim([3500, 9900])
        ax.set_xlabel('$\lambda$ [$\AA$]')
        #ax.set_ylabel('$F_{\lambda}$ [$10^{-17} erg\ s^{-1}\ cm^{-2}\ \AA^{-1$]')
        
        # Save the current plot to the PDF file
        pdf.savefig(fig)
        plt.close()

# After the loop, all the plots will be saved in the PDF file
print(f'All plots saved in {pdf_filename}')


['sv3' 'sv3' 'sv3' 'sv3' 'sv3' 'sv3' 'sv3' 'sv3' 'sv1'] ['bright' 'bright' 'dark' 'bright' 'dark' 'bright' 'bright' 'bright'
 'other'] [11520 11520 11520 11520 11520 11520 11520 11520 11520] [115 115 115 115 115 115 115 115 115] [ 0.10812171  0.30911242  0.52981973  0.3420389   0.96338491  0.21460966
  0.4195996   0.42451453 -0.0019957 ]
39633322455860934
coadd-sv3-bright-11520.fits already exists in DESI_Downloads. No need to download.
INFO:spectra.py:372:read_spectra: iotime 1.574 sec to read coadd-sv3-bright-11520.fits at 2023-11-03T15:50:50.152211
39633318970393119
coadd-sv3-bright-11520.fits already exists in DESI_Downloads. No need to download.
INFO:spectra.py:372:read_spectra: iotime 1.577 sec to read coadd-sv3-bright-11520.fits at 2023-11-03T15:50:52.321466
39633325920356118
coadd-sv3-dark-11520.fits already exists in DESI_Downloads. No need to download.
INFO:spectra.py:372:read_spectra: iotime 1.789 sec to read coadd-sv3-dark-11520.fits at 2023-11-03T15:50:54.580791
3963331897